In [1]:
!pip show protobuf

Name: protobuf
Version: 3.20.3
Summary: Protocol Buffers
Home-page: https://developers.google.com/protocol-buffers/
Author: 
Author-email: 
License: BSD-3-Clause
Location: c:\users\nisarg\anaconda3\lib\site-packages
Requires: 
Required-by: tensorflow, tensorboard, proto-plus, googleapis-common-protos, google-api-core


In [1]:
import pandas as pd
import tensorflow as tf
print("TF:", tf.__version__)

TF: 2.8.0


In [2]:
PATH = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
PATH_test = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"
COLUMNS = ['age','workclass', 'fnlwgt', 'education', 'education_num', 'marital', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_week', 'native_country', 'label']

In [3]:
df_train = pd.read_csv(PATH, skipinitialspace=True, names = COLUMNS, index_col=False)
df_test = pd.read_csv(PATH_test, 
                     skiprows=1, 
                     skipinitialspace=True, 
                     names=COLUMNS, 
                     index_col=False, 
                     engine='python',
                      na_values="?")
df_test['label'] = df_test['label'].str.replace('.', '', regex=False)

In [4]:
print(df_train.shape, df_test.shape)

(32561, 15) (16281, 15)


In [7]:
print(df_train.dtypes)

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital           object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_week         int64
native_country    object
label              int64
dtype: object


In [6]:
label_map_train = {'<=50K': 0, '>50K': 1}
label_map_test = {'<=50K.': 0, '>50K.': 1}

df_train['label'] = df_train['label'].map(label_map_train)
df_test['label'] = df_test['label'].map(label_map_test)

In [8]:
print("Training label counts:\n", df_train['label'].value_counts())
print("\nTest label counts:\n", df_test['label'].value_counts())

Training label counts:
 0    24720
1     7841
Name: label, dtype: int64

Test label counts:
 Series([], Name: label, dtype: int64)


In [9]:
print(df_train.dtypes)

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital           object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_week         int64
native_country    object
label              int64
dtype: object


In [10]:
CONTI_FEATURES = ['age', 'fnlwgt','capital_gain', 'education_num', 'capital_loss', 'hours_week']
### Define the categorical list
CATE_FEATURES = ['workclass', 'education', 'marital', 'occupation', 'relationship', 'race', 'sex', 'native_country']

In [11]:
continuous_features = [tf.feature_column.numeric_column(k) for k in CONTI_FEATURES]

In [12]:
relationship = tf.feature_column.categorical_column_with_vocabulary_list('relationship', ['Husband', 'Not-in-family', 'Wife', 'Own-child', 'Unmarried', 'Other-relative'])

In [13]:
categorical_features = [tf.feature_column.categorical_column_with_hash_bucket(k, hash_bucket_size=1000) for k in CATE_FEATURES]

In [14]:
model = tf.estimator.LinearClassifier(n_classes=2, model_dir="ongoing/train", feature_columns=categorical_features + continuous_features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'ongoing/train', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
FEATURES = ['age','workclass', 'fnlwgt', 'education', 'education_num', 'marital', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_week', 'native_country', 'label']
LABEL = 'label'
def get_input_fn(data_set, num_epochs=None, n_batch=128, shuffle=True):
    # Use tf.compat.v1 to access the inputs module in TF 2.x
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
       x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
       y=pd.Series(data_set[LABEL].values),
       batch_size=n_batch,
       num_epochs=num_epochs,
       shuffle=shuffle)

In [16]:
model = tf.estimator.LinearClassifier(
    n_classes=2, 
    model_dir="train_output", # Simpler folder name
    feature_columns=categorical_features + continuous_features
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
 model.train(input_fn=get_input_fn(df_train, num_epochs=None, n_batch=128, shuffle=False), steps=1000)



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.


C:\Users\nisarg\anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train_output\model.ckpt-1000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 1000...
INFO:tensorflow:Saving checkpoints for 1000 into train_output\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 1000...
INFO:tensorflow:loss = 705.4092, step = 1000
INFO:tensorflow:global_step/sec: 244.886
INFO:tensorflow:loss = 178.27924, step = 1100 (0.408 sec)
INFO:tensorflow:global_step/sec: 460.731
INFO:tensorflow:loss = 144.58835, step = 

In [19]:
model.evaluate(
    input_fn=get_input_fn(df_test, num_epochs=1, n_batch=128, shuffle=False),
    steps=1000
)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2026-01-12T20:07:08
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train_output\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InternalError'>, Graph execution error:

Unsupported object type float


InternalError: Graph execution error:

Unsupported object type float

In [20]:
print(df_test.dtypes)

# show only object columns (most likely troublemakers)
obj_cols = df_test.select_dtypes(include=["object"]).columns.tolist()
print("Object columns:", obj_cols)

# peek at a few values from those columns
for c in obj_cols[:20]:
    print("\n", c)
    print(df_test[c].head(5))

age                 int64
workclass          object
fnlwgt              int64
education          object
education_num       int64
marital            object
occupation         object
relationship       object
race               object
sex                object
capital_gain        int64
capital_loss        int64
hours_week          int64
native_country     object
label             float64
dtype: object
Object columns: ['workclass', 'education', 'marital', 'occupation', 'relationship', 'race', 'sex', 'native_country']

 workclass
0      Private
1      Private
2    Local-gov
3      Private
4          NaN
Name: workclass, dtype: object

 education
0            11th
1         HS-grad
2      Assoc-acdm
3    Some-college
4    Some-college
Name: education, dtype: object

 marital
0         Never-married
1    Married-civ-spouse
2    Married-civ-spouse
3    Married-civ-spouse
4         Never-married
Name: marital, dtype: object

 occupation
0    Machine-op-inspct
1      Farming-fishing
2      Pro

In [25]:
df_train["label"] = df_train["label"].astype(int)
df_test["label"]  = df_test["label"].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [26]:
import numpy as np

print("dtype:", df_test["label"].dtype)
print("NaNs:", df_test["label"].isna().sum())
print("Infs:", np.isinf(df_test["label"]).sum())
print("Unique (first 20):", df_test["label"].dropna().unique()[:20])

dtype: float64
NaNs: 16281
Infs: 0
Unique (first 20): []


In [27]:
import numpy as np

df_test = df_test.dropna(subset=["label"]).copy()
print("Rows after dropping NaN labels:", len(df_test))
print("Label dtype now:", df_test["label"].dtype)
print("Unique labels:", df_test["label"].unique()[:20])

Rows after dropping NaN labels: 0
Label dtype now: float64
Unique labels: []


In [28]:
print("df_train label NaNs:", df_train["label"].isna().sum())
print("df_train rows:", len(df_train))
print("df_train label unique (first 10):", df_train["label"].dropna().unique()[:10])

df_train label NaNs: 0
df_train rows: 32561
df_train label unique (first 10): [0 1]


In [29]:
df_train2 = df_train.sample(frac=0.8, random_state=42)
df_valid  = df_train.drop(df_train2.index)

print("df_train2 rows:", len(df_train2), "label NaNs:", df_train2["label"].isna().sum())
print("df_valid rows:", len(df_valid), "label NaNs:", df_valid["label"].isna().sum())
print("valid label counts:\n", df_valid["label"].value_counts())

df_train2 rows: 26049 label NaNs: 0
df_valid rows: 6512 label NaNs: 0
valid label counts:
 0    4944
1    1568
Name: label, dtype: int64


In [30]:
cat_cols = ['workclass','education','marital','occupation','relationship','race','sex','native_country']

for c in cat_cols:
    df_train2[c] = df_train2[c].fillna("Unknown").astype(str)
    df_valid[c]  = df_valid[c].fillna("Unknown").astype(str)

print("Any NaNs left in categorical columns (train2):", df_train2[cat_cols].isna().sum().sum())
print("Any NaNs left in categorical columns (valid):",  df_valid[cat_cols].isna().sum().sum())
print("Sample workclass values:", df_train2["workclass"].head(5).tolist())

Any NaNs left in categorical columns (train2): 0
Any NaNs left in categorical columns (valid): 0
Sample workclass values: ['Private', 'State-gov', 'Private', 'Private', 'Self-emp-not-inc']


In [31]:
import tensorflow as tf

num_cols = ["age","fnlwgt","education_num","capital_gain","capital_loss","hours_week"]
cat_cols = ['workclass','education','marital','occupation','relationship','race','sex','native_country']

feature_columns = []

# numeric columns
for c in num_cols:
    feature_columns.append(tf.feature_column.numeric_column(c))

# categorical columns with vocab from training data
for c in cat_cols:
    vocab = df_train2[c].unique().tolist()
    cat = tf.feature_column.categorical_column_with_vocabulary_list(
        key=c,
        vocabulary_list=vocab,
        num_oov_buckets=1
    )
    feature_columns.append(tf.feature_column.indicator_column(cat))

print("Feature columns built:", len(feature_columns))


Feature columns built: 14


In [32]:
def get_input_fn(df, label_col="label", num_epochs=1, n_batch=128, shuffle=False):
    y = df[label_col].astype(int)
    X = df.drop(columns=[label_col])
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
        x=X,
        y=y,
        batch_size=n_batch,
        num_epochs=num_epochs,
        shuffle=shuffle
    )

model = tf.estimator.LinearClassifier(feature_columns=feature_columns)

print("Model created.")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\nisarg\\AppData\\Local\\Temp\\tmpk3toze1o', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Model created.


In [33]:
model.train(
    input_fn=get_input_fn(df_train2, num_epochs=None, n_batch=128, shuffle=True),
    steps=1000
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\nisarg\AppData\Local\Temp\tmpk3toze1o\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 316.978
INFO:tensorflow:loss = 1080.9664, step = 100 (0.317 sec)
INFO:tensorflow:global_step/sec: 444.301
INFO:tensorflow:loss = 988.7019, step = 200 (0.229 sec)
INFO:tensorflow:global_step/sec: 447.512
INFO:tensorflow:loss = 218.15543, step = 300 (0.219 sec)
INFO:tensorflow:global_step/sec: 464.197
INFO:tensorflow:loss = 244.1971, step = 400 (0.217 sec)
INFO:tensorflow:global_step/sec: 444.294
INFO:tensorflow:loss = 410.64606, s

In [34]:
eval_steps = len(df_valid) // 128
eval_steps


50

In [35]:
model.evaluate(
    input_fn=get_input_fn(df_valid, num_epochs=1, n_batch=128, shuffle=False),
    steps=50
)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2026-01-12T20:18:03
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\nisarg\AppData\Local\Temp\tmpk3toze1o\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [5/50]
INFO:tensorflow:Evaluation [10/50]
INFO:tensorflow:Evaluation [15/50]
INFO:tensorflow:Evaluation [20/50]
INFO:tensorflow:Evaluation [25/50]
INFO:tensorflow:Evaluation [30/50]
INFO:tensorflow:Evaluation [35/50]
INFO:tensorflow:Evaluation [40/50]
INFO:tensorflow:Evaluation [45/50]
INFO:tensorflow:Evaluation [50/50]
INFO:tensorflow:Inference Time : 1.04450s
INFO:tensorflow:Finished evaluation at 2026-01-12-20:18:05
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.2409375, accuracy_baseline = 0.7590625, auc = 0.5, auc_precision_recall = 0.2409375, average_loss = 176.79425, global_step = 1000,

{'accuracy': 0.2409375,
 'accuracy_baseline': 0.7590625,
 'auc': 0.5,
 'auc_precision_recall': 0.2409375,
 'average_loss': 176.79425,
 'label/mean': 0.2409375,
 'loss': 176.79425,
 'precision': 0.2409375,
 'prediction/mean': 1.0,
 'recall': 1.0,
 'global_step': 1000}

In [36]:
# salary is 0 at a very young age and then keeps increasing.
# Close to retirement it again decreases. hence we square age
def square_var(df_t, df_te, var_name='age'):
    df_t['new'] = df_t[var_name].pow(2)
    df_te['new'] = df_te[var_name].pow(2)
    return df_t, df_te


In [37]:
df_train_new, df_test_new, = square_var(df_train, df_test, var_name = 'age')

In [38]:
print(df_train_new.shape, df_test_new.shape)

(32561, 16) (0, 16)


In [39]:
CONTI_FEATURES_NEW = [
    'age',
    'fnlwgt',
    'capital_gain',
    'education_num',
    'capital_loss',
    'hours_week',
    'new'
]

continuous_features_new = [
    tf.feature_column.numeric_column(k)
    for k in CONTI_FEATURES_NEW
]


In [40]:
model_1 = tf.estimator.LinearClassifier(
    model_dir="ongoing/train1",
    feature_columns=categorical_features + continuous_features_new
)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'ongoing/train1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [44]:
FEATURES_NEW = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital',
    'occupation', 'relationship', 'race', 'sex',
    'capital_gain', 'capital_loss', 'hours_week', 'native_country', 'new'
]

def get_input_fn(data_set, num_epochs=None, n_batch=128, shuffle=True):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
        x=pd.DataFrame({k: data_set[k].values for k in FEATURES_NEW}),
        y=pd.Series(data_set[LABEL].values),
        batch_size=n_batch,
        num_epochs=num_epochs,
        shuffle=shuffle
    )

In [45]:
model_1.train(
    input_fn=get_input_fn(
        df_train,
        num_epochs=None,
        n_batch=128,
        shuffle=False
    ),
    steps=1000
)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into ongoing/train1\model.ckpt.


NotFoundError: Graph execution error:

Detected at node 'save/SaveV2' defined at (most recent call last):
    File "C:\Users\nisarg\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\nisarg\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\nisarg\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\nisarg\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\nisarg\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\nisarg\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\nisarg\anaconda3\lib\asyncio\base_events.py", line 596, in run_forever
      self._run_once()
    File "C:\

In [46]:
import os

os.makedirs("ongoing/train1", exist_ok=True)
print("Exists?", os.path.exists("ongoing/train1"))
print("Full path:", os.path.abspath("ongoing/train1"))

Exists? True
Full path: C:\Users\nisarg\ongoing\train1


In [47]:
model_1.train(
    input_fn=get_input_fn(df_train, num_epochs=None, n_batch=128, shuffle=False),
    steps=1000
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into ongoing/train1\model.ckpt.


NotFoundError: Graph execution error:

Detected at node 'save/SaveV2' defined at (most recent call last):
    File "C:\Users\nisarg\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\nisarg\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\nisarg\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\nisarg\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\nisarg\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\nisarg\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\nisarg\anaconda3\lib\asyncio\base_events.py", line 596, in run_forever
      self._run_once()
    File "C:\

In [48]:
import os

print("CWD:", os.getcwd())

MODEL_DIR = os.path.abspath("ongoing/train1")
os.makedirs(MODEL_DIR, exist_ok=True)

# Create the temp checkpoint directory TF is trying to use at step 0
os.makedirs(os.path.join(MODEL_DIR, "model.ckpt-0_temp"), exist_ok=True)

print("MODEL_DIR:", MODEL_DIR)
print("Temp dir exists?:", os.path.exists(os.path.join(MODEL_DIR, "model.ckpt-0_temp")))


CWD: C:\Users\nisarg
MODEL_DIR: C:\Users\nisarg\ongoing\train1
Temp dir exists?: True


In [49]:
model_1 = tf.estimator.LinearClassifier(
    model_dir=MODEL_DIR,
    feature_columns=categorical_features + continuous_features_new
)
print("model_1 model_dir:", model_1.model_dir)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\nisarg\\ongoing\\train1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
model_1 model_dir: C:\Users\nisarg\ongoing\train1


In [50]:
model_1.train(
    input_fn=get_input_fn(df_train, num_epochs=None, n_batch=128, shuffle=False),
    steps=1000
)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\nisarg\ongoing\train1\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 166.734
INFO:tensorflow:loss = 606.06586, step = 100 (0.598 sec)
INFO:tensorflow:global_step/sec: 299.768
INFO:tensorflow:loss = 260.04898, step = 200 (0.330 sec)
INFO:tensorflow:global_step/sec: 317.798
INFO:tensorflow:loss = 838.8059, step = 300 (0.317 sec)
INFO:tensorflow:global_step/sec: 298.264
INFO:tensorflow:loss = 97.0002, step = 400 (0.335 sec)
INFO:tensorflow:global_step/sec: 375.836
INFO:tensorflow:loss = 119.83976, step = 500 (0.265 

In [51]:
eval_steps = len(df_valid) // 128
eval_steps


50

In [52]:
model_1.evaluate(
    input_fn=get_input_fn(df_valid, num_epochs=1, n_batch=128, shuffle=False),
    steps=50
)


KeyError: 'new'

In [53]:
df_train["new"] = df_train["age"].pow(2)
df_valid["new"] = df_valid["age"].pow(2)

print("new in df_train?", "new" in df_train.columns)
print("new in df_valid?", "new" in df_valid.columns)
print(df_valid[["age", "new"]].head())


new in df_train? True
new in df_valid? True
    age   new
9    42  1764
11   30   900
13   32  1024
16   25   625
24   59  3481


In [54]:
model_1.evaluate(
    input_fn=get_input_fn(df_valid, num_epochs=1, n_batch=128, shuffle=False),
    steps=50
)


INFO:tensorflow:Calling model_fn.


C:\Users\nisarg\anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2026-01-12T20:39:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\nisarg\ongoing\train1\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [5/50]
INFO:tensorflow:Evaluation [10/50]
INFO:tensorflow:Evaluation [15/50]
INFO:tensorflow:Evaluation [20/50]
INFO:tensorflow:Evaluation [25/50]
INFO:tensorflow:Evaluation [30/50]
INFO:tensorflow:Evaluation [35/50]
INFO:tensorflow:Evaluation [40/50]
INFO:tensorflow:Evaluation [45/50]
INFO:tensorflow:Evaluation [50/50]
INFO:tensorflow:Inference Time : 1.32909s
INFO:tensorflow:Finished evaluation at 2026-01-12-20:39:08
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.78234375, accuracy_baseline = 0.7590625, auc = 0.5548316, auc_precision_recall = 0.3594716, average_loss = 427.74963, global_step = 1000, label/mean = 0.2409375, loss = 427.74963, 

{'accuracy': 0.78234375,
 'accuracy_baseline': 0.7590625,
 'auc': 0.5548316,
 'auc_precision_recall': 0.3594716,
 'average_loss': 427.74963,
 'label/mean': 0.2409375,
 'loss': 427.74963,
 'precision': 0.86341465,
 'prediction/mean': 0.032015268,
 'recall': 0.11478599,
 'global_step': 1000}

In [55]:
model_1.evaluate(
    input_fn=get_input_fn(df_test_new, num_epochs=1, n_batch=128, shuffle=False), 
    steps=1000
)

ZeroDivisionError: integer division or modulo by zero

In [56]:
print("df_test_new shape:", df_test_new.shape)
print("df_test_new label NaNs:", df_test_new["label"].isna().sum() if "label" in df_test_new.columns else "NO label column")
print("Columns:", df_test_new.columns.tolist())


df_test_new shape: (0, 16)
df_test_new label NaNs: 0
Columns: ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_week', 'native_country', 'label', 'new']


In [57]:
print("df_test shape:", df_test.shape)
print("df_test label NaNs:", df_test["label"].isna().sum() if "label" in df_test.columns else "NO label column")
print("df_test label dtype:", df_test["label"].dtype if "label" in df_test.columns else "NA")


df_test shape: (0, 16)
df_test label NaNs: 0
df_test label dtype: float64


In [58]:
[x for x in globals().keys() if "test" in x.lower()]

['PATH_test', 'df_test', 'label_map_test', 'X_test', 'df_test_new']

In [59]:
print("PATH_test =", PATH_test)

df_test = pd.read_csv(PATH_test)
print("Reloaded df_test shape:", df_test.shape)
print("Columns:", df_test.columns.tolist())
print("Label NaNs:", df_test["label"].isna().sum() if "label" in df_test.columns else "NO label column")


PATH_test = https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test
Reloaded df_test shape: (16281, 1)
Columns: ['|1x3 Cross validator']
Label NaNs: NO label column


In [60]:
cols = [
    "age","workclass","fnlwgt","education","education_num","marital","occupation",
    "relationship","race","sex","capital_gain","capital_loss","hours_week",
    "native_country","income"
]

df_test = pd.read_csv(
    PATH_test,
    sep=r",\s*",
    engine="python",
    header=0,          # adult.test has a header/comment line
    names=cols
)

print("df_test shape:", df_test.shape)
print(df_test.head())
print("income NaNs:", df_test["income"].isna().sum())
print("Unique income (first 5):", df_test["income"].dropna().unique()[:5])


ValueError: Number of passed names did not match number of header fields in the file

In [61]:
cols = [
    "age","workclass","fnlwgt","education","education_num","marital","occupation",
    "relationship","race","sex","capital_gain","capital_loss","hours_week",
    "native_country","income"
]

df_test = pd.read_csv(
    PATH_test,
    sep=r",\s*",
    engine="python",
    header=None,
    names=cols,
    skiprows=1
)

print("df_test shape:", df_test.shape)
print(df_test.head())
print("income NaNs:", df_test["income"].isna().sum())
print("Unique income (first 5):", df_test["income"].dropna().unique()[:5])


df_test shape: (16281, 15)
   age  workclass  fnlwgt     education  education_num             marital  \
0   25    Private  226802          11th              7       Never-married   
1   38    Private   89814       HS-grad              9  Married-civ-spouse   
2   28  Local-gov  336951    Assoc-acdm             12  Married-civ-spouse   
3   44    Private  160323  Some-college             10  Married-civ-spouse   
4   18          ?  103497  Some-college             10       Never-married   

          occupation relationship   race     sex  capital_gain  capital_loss  \
0  Machine-op-inspct    Own-child  Black    Male             0             0   
1    Farming-fishing      Husband  White    Male             0             0   
2    Protective-serv      Husband  White    Male             0             0   
3  Machine-op-inspct      Husband  Black    Male          7688             0   
4                  ?    Own-child  White  Female             0             0   

   hours_week native_co

In [62]:
# 1) Map income -> label (strip spaces)
df_test["income"] = df_test["income"].astype(str).str.strip()
df_test["label"] = (df_test["income"] == ">50K.").astype(int)

# 2) Create squared age feature
df_test["new"] = df_test["age"].pow(2)

# 3) Clean categorical columns: replace '?' with 'Unknown'
cat_cols = ['workclass','education','marital','occupation','relationship','race','sex','native_country']
for c in cat_cols:
    df_test[c] = df_test[c].replace("?", "Unknown").fillna("Unknown").astype(str)

print("df_test shape:", df_test.shape)
print("label counts:\n", df_test["label"].value_counts())
print(df_test[["age","new","income","label"]].head())


df_test shape: (16281, 17)
label counts:
 0    12435
1     3846
Name: label, dtype: int64
   age   new  income  label
0   25   625  <=50K.      0
1   38  1444  <=50K.      0
2   28   784   >50K.      1
3   44  1936   >50K.      1
4   18   324  <=50K.      0


In [63]:
test_steps = len(df_test) // 128
test_steps


127

In [64]:
model_1.evaluate(
    input_fn=get_input_fn(
        df_test,
        num_epochs=1,
        n_batch=128,
        shuffle=False
    ),
    steps=127
)


INFO:tensorflow:Calling model_fn.


C:\Users\nisarg\anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2026-01-12T20:47:00
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\nisarg\ongoing\train1\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [12/127]
INFO:tensorflow:Evaluation [24/127]
INFO:tensorflow:Evaluation [36/127]
INFO:tensorflow:Evaluation [48/127]
INFO:tensorflow:Evaluation [60/127]
INFO:tensorflow:Evaluation [72/127]
INFO:tensorflow:Evaluation [84/127]
INFO:tensorflow:Evaluation [96/127]
INFO:tensorflow:Evaluation [108/127]
INFO:tensorflow:Evaluation [120/127]
INFO:tensorflow:Evaluation [127/127]
INFO:tensorflow:Inference Time : 1.46396s
INFO:tensorflow:Finished evaluation at 2026-01-12-20:47:02
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.78537154, accuracy_baseline = 0.763595, auc = 0.55405885, auc_precision_recall = 0.35065007, average_loss = 424.1556, global_step 

{'accuracy': 0.78537154,
 'accuracy_baseline': 0.763595,
 'auc': 0.55405885,
 'auc_precision_recall': 0.35065007,
 'average_loss': 424.1556,
 'label/mean': 0.23640501,
 'loss': 424.1556,
 'precision': 0.83396226,
 'prediction/mean': 0.032585993,
 'recall': 0.115014315,
 'global_step': 1000}

In [65]:
predictions = list(model_1.predict(input_fn=get_input_fn(df_test_new, num_epochs=1, n_batch=128, shuffle=False)))

ZeroDivisionError: integer division or modulo by zero